In [ ]:
### Import libraries
from netCDF4 import Dataset
import xarray as xr
import numpy as np
import cartopy.crs as crs
import matplotlib
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from wrf import getvar, interplevel, smooth2d
import warnings
from scipy.interpolate import griddata
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
warnings.filterwarnings('ignore')
import matplotlib.transforms as mtransforms
import matplotlib.ticker as mticker

CP = 1005.7
RD = 287.04
P0 = 1000.
TR = 300.
LV = 2.501e6
EPS = 1.


In [ ]:
### Parameters
nx = 400
ny = 250
nz = 44

### Read in the data
ctl = Dataset('/p/work1/lloveras/nov2018/30km_files/ctl/wrfout_d01_2018-11-13_12_00_00')
adj = Dataset('/p/work1/lloveras/nov2018/30km_files/adj_full/wrfout_d01_2018-11-13_12_00_00')

### Latitude and longitude
lats = np.asarray(getvar(ctl,'lat'))
lons = np.asarray(getvar(ctl,'lon'))
lons[lons > 0] -= 360

### Pressure
p_ctl = np.asarray(getvar(ctl,'pressure'))
p_adj = np.asarray(getvar(adj,'pressure'))

### Height
z_ctl = np.asarray(getvar(ctl,'z'))

### Theta
t_ctl = np.asarray(getvar(ctl,'THM'))
t_adj = np.asarray(getvar(adj,'THM'))

### Levels
z850 = interplevel(z_ctl, p_ctl, 850)
dt850 = interplevel(t_adj, p_adj, 850) - interplevel(t_ctl, p_ctl, 850)

z700 = interplevel(z_ctl, p_ctl, 700)
dt700 = interplevel(t_adj, p_adj, 700) - interplevel(t_ctl, p_ctl, 700)

z600 = interplevel(z_ctl, p_ctl, 600)
dt600 = interplevel(t_adj, p_adj, 600) - interplevel(t_ctl, p_ctl, 600)

z500 = interplevel(z_ctl, p_ctl, 500)
dt500 = interplevel(t_adj, p_adj, 500) - interplevel(t_ctl, p_ctl, 500)

z400 = interplevel(z_ctl, p_ctl, 400)
dt400 = interplevel(t_adj, p_adj, 400) - interplevel(t_ctl, p_ctl, 400)

z300 = interplevel(z_ctl, p_ctl, 300)
dt300 = interplevel(t_adj, p_adj, 300) - interplevel(t_ctl, p_ctl, 300)


In [ ]:
### Contour interval
zlvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])/5

### Plot subdomain
min_lat = 15
max_lat = 62
min_lon = -130
max_lon = -70

### Cartopy projection
proj = crs.LambertConformal(central_longitude=-100,standard_parallels=(20,70))

fig, axd = plt.subplot_mosaic([['topleft','topmiddle','topright'],['bottomleft','bottommiddle','bottomright'],['cbar','cbar','cbar']],
                              constrained_layout=True, figsize=(8,5), dpi=200, 
                              gridspec_kw={'width_ratios':[1,1,1],'height_ratios':[1,1,0.05]},
                              per_subplot_kw={'topleft':{'projection':proj},
                                              'topmiddle':{'projection':proj},
                                              'topright':{'projection':proj},
                                              'bottomleft':{'projection':proj},
                                              'bottommiddle':{'projection':proj},
                                              'bottomright':{'projection':proj}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### TOP LEFT
#############

# Add the gridlines
gls = axd['topleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=False
gls.rotate_labels = False

cs1 = axd['topleft'].contour(lons, lats, smooth2d(z300,1)/10.,
                  levels=np.arange(0,1500,25),
                  colors='k', transform=crs.PlateCarree())
axd['topleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,25),fontsize=6,colors='k')
im1 = axd['topleft'].contourf(lons, lats, dt300, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topleft'].set_title('300 hPa')
axd['topleft'].text(0.0, 1.0, '(a)',transform=axd['topleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP MIDDLE
#############

gls = axd['topmiddle'].gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topmiddle'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])

cs1 = axd['topmiddle'].contour(lons, lats, smooth2d(z400,1)/10.,
                  levels=np.arange(0,1500,25),
                  colors='k', transform=crs.PlateCarree())
axd['topmiddle'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,25),fontsize=6,colors='k')
im1 = axd['topmiddle'].contourf(lons, lats, dt400, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topmiddle'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topmiddle'].set_title('400 hPa')
axd['topmiddle'].text(0.0, 1.0, '(b)',transform=axd['topmiddle'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP RIGHT
#############

gls = axd['topright'].gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])

cs1 = axd['topright'].contour(lons, lats, smooth2d(z500,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['topright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im1 = axd['topright'].contourf(lons, lats, dt500, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topright'].set_title('500 hPa')
axd['topright'].text(0.0, 1.0, '(c)',transform=axd['topright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM LEFT
#############

gls = axd['bottomleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.rotate_labels = False

cs1 = axd['bottomleft'].contour(lons, lats, smooth2d(z600,1)/10.,
                  levels=np.arange(0,1500,15),
                  colors='k', transform=crs.PlateCarree())
axd['bottomleft'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,15),fontsize=6,colors='k')
im1 = axd['bottomleft'].contourf(lons, lats, dt600, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottomleft'].set_title('600 hPa')
axd['bottomleft'].text(0.0, 1.0, '(d)',transform=axd['bottomleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM MIDDLE
#############

gls = axd['bottommiddle'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottommiddle'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.left_labels=False
gls.rotate_labels = False

cs1 = axd['bottommiddle'].contour(lons, lats, smooth2d(z700,1)/10.,
                  levels=np.arange(0,1500,10),
                  colors='k', transform=crs.PlateCarree())
axd['bottommiddle'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,10),fontsize=6,colors='k')
im1 = axd['bottommiddle'].contourf(lons, lats, dt700, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottommiddle'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottommiddle'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottommiddle'].set_title('700 hPa')
axd['bottommiddle'].text(0.0, 1.0, '(e)',transform=axd['bottommiddle'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM RIGHT
#############

gls = axd['bottomright'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.left_labels=False
gls.rotate_labels = False

cs1 = axd['bottomright'].contour(lons, lats, smooth2d(z850,1)/10.,
                  levels=np.arange(0,1500,5),
                  colors='k', transform=crs.PlateCarree())
axd['bottomright'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,5),fontsize=6,colors='k')
im1 = axd['bottomright'].contourf(lons, lats, dt850, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottomright'].set_title('850 hPa')
axd['bottomright'].text(0.0, 1.0, '(e)',transform=axd['bottomright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### SET THE COLORBAR AND SHOW
cbar = fig.colorbar(im1, orientation='horizontal', cax=axd['cbar'])
plt.savefig('/p/work1/lloveras/nov2018/figs_png/30km_perts_6panels.png',bbox_inches='tight')
plt.show()
